In [1]:
import geopy
import requests
import pandas as pd
from datetime import datetime
from geopy.distance import geodesic
import logging
import pytz
import threading
import time
import sqlite3

# Configuração de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Função para criar o banco de dados e as tabelas, se não existirem
def criar_banco_e_tabelas(nome_banco):
    conn = sqlite3.connect(nome_banco)
    cursor = conn.cursor()

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS onibus (
            COD INTEGER,
            REFRESH TEXT,
            LAT_IN_TIME REAL,
            LON_IN_TIME REAL,
            CODIGOLINHA INTEGER,
            ADAPT INTEGER,
            TIPO_VEIC INTEGER,
            TABELA TEXT,
            SITUACAO TEXT,
            SITUACAO2 TEXT,
            SENT TEXT,
            TCOUNT INTEGER,
            SENTIDO_IN_TIME TEXT,
            HORA TEXT,
            FLAG_PROCES INTEGER
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS onibus_buffer (
            COD INTEGER,
            REFRESH TEXT,
            LAT_IN_TIME REAL,
            LON_IN_TIME REAL,
            CODIGOLINHA INTEGER,
            ADAPT INTEGER,
            TIPO_VEIC INTEGER,
            TABELA TEXT,
            SITUACAO TEXT,
            SITUACAO2 TEXT,
            SENT TEXT,
            TCOUNT INTEGER,
            SENTIDO_IN_TIME TEXT,
            HORA TEXT,
            FLAG_PROCES INTEGER
        )
    ''')

    conn.commit()
    conn.close()

# Função para carregar uma tabela para um DataFrame
def carregar_tabela_para_dataframe(nome_tabela, banco_de_dados):
    conn = sqlite3.connect(banco_de_dados)
    try:
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?;", (nome_tabela,))
        if cursor.fetchone():
            return pd.read_sql_query(f"SELECT * FROM {nome_tabela}", conn)
        return pd.DataFrame()
    finally:
        conn.close()

# Função para buscar e processar dados de uma linha de ônibus
def buscar_e_processar_dados(linha):
    url_base = f'https://transporteservico.urbs.curitiba.pr.gov.br/getVeiculos.php?linha={linha:03}&c=821f0'
    try:
        response = requests.get(url_base, timeout=20)
        response.raise_for_status()
        timezone_sp = pytz.timezone('America/Sao_Paulo')
        hora_online = datetime.now(timezone_sp).strftime("%Y-%m-%d %H:%M:%S")
        dados_json = response.json()

        if not dados_json or not isinstance(dados_json, dict):
            return None

        codigos_onibus = [{
            "COD": valor['COD'], "REFRESH": valor['REFRESH'], "LAT_IN_TIME": valor['LAT'],
            "LON_IN_TIME": valor['LON'], "CODIGOLINHA": int(valor['CODIGOLINHA']),
            "ADAPT": int(valor['ADAPT']), "TIPO_VEIC": int(valor['TIPO_VEIC']),
            "TABELA": valor['TABELA'], "SITUACAO": valor['SITUACAO'],
            "SITUACAO2": valor['SITUACAO2'], "SENT": valor['SENT'],
            "TCOUNT": valor.get('TCOUNT', 0), "SENTIDO_IN_TIME": valor['SENTIDO']
        } for chave, valor in dados_json.items()]

        df_codigos_onibus = pd.DataFrame(codigos_onibus)
        if not df_codigos_onibus.empty:
            df_codigos_onibus['HORA'] = hora_online
            return df_codigos_onibus
    except requests.RequestException:
        return None

# Função para remover duplicatas
def remover_duplicatas():
    with sqlite3.connect('dados_onibus.db') as conn:
        cursor = conn.cursor()
        logging.info("Removendo duplicatas da tabela onibus...")
        cursor.execute('''
            DELETE FROM onibus
            WHERE rowid NOT IN (
                SELECT MIN(rowid)
                FROM onibus
                GROUP BY COD, LAT_IN_TIME, LON_IN_TIME
            )
        ''')
        conn.commit()

# Função para limpar a tabela buffer
def limpar_tabela_buffer(nome_banco, nome_tabela):
    with sqlite3.connect(f'{nome_banco}.db') as conn:
        cursor = conn.cursor()
        cursor.execute(f'DELETE FROM {nome_tabela}')
        conn.commit()

# Função para gravar dados no banco
def gravar_no_banco(df_result, banco_de_dados, tabela):
    try:
        with sqlite3.connect(banco_de_dados) as conn:
            df_result.to_sql(tabela, conn, if_exists='append', index=False)
    except Exception as e:
        logging.error(f"Erro ao gravar no banco de dados: {e}")

# Função para processar uma linha específica
def processar_linha(linha, df_linhas, result_list):
    df_result = buscar_e_processar_dados(linha)
    if df_result is not None:
        result_list.append(df_result)

# Função para processar linhas usando threads
def processar_linhas_com_threads(linhas_unicas, df_linhas):
    global df_concatenado

    result_list = []
    threads = [
        threading.Thread(target=processar_linha, args=(linha, df_linhas, result_list))
        for linha in linhas_unicas
    ]

    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()

    if result_list:
        df_concatenado = pd.concat(result_list, ignore_index=True)
        gravar_no_banco(df_concatenado, 'dados_onibus.db', 'onibus')
        limpar_tabela_buffer('dados_onibus', 'onibus_buffer')
        gravar_no_banco(df_concatenado, 'dados_onibus.db', 'onibus_buffer')
        remover_duplicatas()

# Inicialização
criar_banco_e_tabelas('dados_onibus.db')


df_linhas = carregar_tabela_para_dataframe('onibus', 'dados_pontos.db')
df_linhas['COD'] = df_linhas['COD'].astype(int)
linhas_unicas = df_linhas['COD'].unique()

df_concatenado = pd.DataFrame()

# Processar linhas
processar_linhas_com_threads(linhas_unicas, df_linhas)


2024-11-05 22:15:16,914 - INFO - Removendo duplicatas da tabela onibus...


In [4]:
df = carregar_tabela_para_dataframe('onibus', 'dados_processados.db')

In [5]:
df

,COD,REFRESH,LAT_IN_TIME,LON_IN_TIME,CODIGOLINHA,ADAPT,TIPO_VEIC,TABELA,SITUACAO,SITUACAO2,SENT,TCOUNT,SENTIDO_IN_TIME,HORA,FLAG_PROCES,PARADA_MAIS_PROXIMA,DISTANCIA_MINIMA,SEQ,SEQ_MAX,ITINERARIO
0,GE722,16:23,-25.504566,-49.282341,302,0,5,,,,,1,sem tabela,2024-10-23 16:23:28,0,inicializado,0.000000,-1,-1,NaN
1,DE720,16:21,-25.429833,-49.277975,302,0,5,,,,,1,sem tabela,2024-10-23 16:23:28,0,inicializado,0.000000,-1,-1,NaN
2,BI877,16:22,-25.399003,-49.246898,386,1,7,,,,,1,sem tabela,2024-10-23 16:23:28,0,inicializado,0.000000,-1,-1,NaN
3,BC925,16:19,-25.393818,-49.241856,386,1,7,,,,,1,sem tabela,2024-10-23 16:23:28,0,inicializado,0.000000,-1,-1,NaN
4,BC010,16:23,-25.382566,-49.288068,171,1,1,1,NO HORÁRIO,REALIZANDO ROTA,VOLTA,1,234-BAIRRO PRIMAVERA (16:26),2024-10-23 16:23:28,0,"Rua Domingos Antônio Moro, 886 - Pilarzinho",26.311335,22,23,8655.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83089,HL495,13:13,-25.435136,-49.273281,707,1,3,1,NO HORÁRIO,REALIZANDO ROTA,IDA,1,672-PRACA RUI BARBOSA (13:16),2024-10-26 13:13:47,0,Praça Rui Barbosa - 707 Tatuquara / Centro,17.519772,5,5,15115.0
83090,HL494,13:13,-25.456541,-49.250746,707,1,3,2,NO HORÁRIO,REALIZANDO ROTA,IDA,1,672-PRACA RUI BARBOSA (13:34),2024-10-26 13:13:47,0,Estação Tubo PUC,397.625065,3,5,15115.0
83091,HL498,13:13,-25.525633,-49.297061,707,1,3,5,ATRASADO,REALIZANDO ROTA,VOLTA,1,1690-TERMINAL TATUQUARA (13:34),2024-10-26 13:13:47,0,Estação Xaxim,3834.115065,4,5,15116.0
83092,JL320,13:13,-25.525990,-49.297221,707,1,3,4,ATRASADO,REALIZANDO ROTA,VOLTA,1,1690-TERMINAL TATUQUARA (13:16),2024-10-26 13:13:47,0,Estação Xaxim,3876.014329,4,5,15116.0
